# Preprocessing: BJH041


This markdown files loads the clean data and does a bipolar rereference as well as epochs the data


In [1]:
import matplotlib
matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np
from scipy import signal, stats
import mat73
import re
from neurodsp.timefrequency import compute_wavelet_transform
import os
import mne
import IPython
import seaborn as sns

In [2]:
## Prep paths ##

subject = 'BJH041'
raw_data_dir = f"/home/brooke/pacman/raw_data/{subject}"
preproc_data_dir = f"/home/brooke/pacman/preprocessing/{subject}/ieeg"

In [3]:
## Load Data ##

# load filtered data #
filtered_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_notched_filtered_clean_ieeg.fif")

# load raw data #
raw_clean_fif = mne.io.Raw(f"{raw_data_dir}/ieeg/{subject}_raw_clean_ieeg.fif")



Opening raw data file /home/brooke/pacman/raw_data/BJH041/ieeg/BJH041_notched_filtered_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH041/ieeg/BJH041_notched_filtered_clean_ieeg-1.fif...
    Range : 2086000 ... 2998199 =   1043.000 ...  1499.100 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH041/ieeg/BJH041_raw_clean_ieeg.fif...
    Range : 0 ... 2085999 =      0.000 ...  1042.999 secs
Ready.
Opening raw data file /home/brooke/pacman/raw_data/BJH041/ieeg/BJH041_raw_clean_ieeg-1.fif...
    Range : 2086000 ... 2998199 =   1043.000 ...  1499.100 secs
Ready.


## Bipolar Rereferencing

In [4]:
# helper functions

def probe_and_num(elec_str): 
    ''' This convenience function takes an electrode string like LHH1 and outputs ('LHH', 1)
    '''
    if '_' in elec_str:
        return None, None
    else:   
        regex_str = '(\D+)(\d+)' # group of letters followed by group of digits
        matcher = re.compile(regex_str,re.IGNORECASE|re.DOTALL).search(elec_str)
        if matcher:
            probe, num = matcher.groups()
            return probe, int(num)
        else:
            return None, None
    
def find_bipolar_pair(ch, labels, remove):
    ''' Find the pair of a given electrode for bipolar referencing.
        Given a single *ch* and a list of *labels*, some of which you want to *remove*,
        this finds the next channel on that probe that isn't meant to be removed (WM is ok).
    '''
    bipolar_pair = None
    probe, num = probe_and_num(ch)
    other_in_probe = [other_ch for other_ch in labels if probe_and_num(other_ch)[0]==probe]
    for i in range(len(other_in_probe)-num):
        next_ch = probe + str(num+i+1)
        if next_ch in remove or next_ch == 'STI':
            continue
        else:
            bipolar_pair = next_ch
            break
    return bipolar_pair

In [5]:
# Here I iterate through the electrodes in my ROIs and match them up with their bipolar pair
pairs = []
anode = []
cathode = []
pairs_name = []
pairs_map = {}
remove = filtered_clean_fif.info['bads']
labels = filtered_clean_fif.info['ch_names']
for ch in labels:
    # if a channel is meant to be removed, it doesn't get to be in a bipolar pair
    if ch in remove or ch == 'STI':
        print(f"{ch} noref")
    else:
        pair = find_bipolar_pair(ch, labels, remove)
        if pair:
            anode.append(ch)
            cathode.append(pair)
            pairs.append((ch, pair))
            pairs_map[ch] = f"{ch}-{pair}"
            pairs_name.append(f"{ch}-{pair}")
            print(ch, pair)

AL1  AL2
AL2 AL3
AL3 AL4
AL4 AL5
REF1 noref
REF2 noref
AL5 AL6
AL6 AL7
AL7 AL8
AL8 AL9
AL9 AL10
AL10 AL11
AL11 AL12
AL12 AL13
AL13 AL14
BL1 BL2
BL2 BL3
BL3 BL4
BL4 BL5
BL5 BL6
BL6 BL7
BL7 BL8
BL8 BL9
BL9 BL10
BL10 BL11
BL11 BL12
BL12 BL13
BL13 BL14
CL1 CL2
CL2 CL3
CL3 CL4
CL4 CL6
CL5 noref
CL6 CL7
CL7 CL8
DL1 DL2
DL2 DL3
DL3 DL4
DL4 DL5
DL5 DL6
DL6 DL7
DL7 DL8
DL8 DL9
DL9 DL10
DL10 DL11
DL11 DL12
DL12 DL13
DL13 DL14
DL14 DL15
DL15 DL16
EL1 noref
EL2 noref
EL3 noref
EL4 noref
EL5 noref
EL6 noref
EL7 noref
EL8 noref
EL9 noref
EL10 noref
FL1 noref
FL2 noref
FL3 noref
FL4 noref
FL5 noref
FL6 noref
FL7 noref
FL8 noref
FL9 noref
FL10 noref
GL1 GL2
GL2 GL3
GL3 GL4
GL4 GL5
GL5 GL6
GL6 GL7
GL7 GL8
GL8 GL9
GL9 GL10
GL10 GL11
GL11 GL12
GL12 GL13
GL13 GL14
GL14 GL15
GL15 GL16
HL1 HL2
HL2 HL3
HL3 HL4
HL4 HL5
HL5 HL6
HL6 HL7
HL7 HL8
HL8 HL9
HL9 HL10
HL10 HL11
HL11 HL12
IL1 noref
IL2 noref
IL3 noref
IL4 noref
IL5 noref
IL6 noref
IL7 noref
IL8 IL9
IL9 IL10
IL10 IL11
IL11 IL12
IL12 IL13
IL13 IL14
IL14 

In [6]:
## Apply Rereference #

if filtered_clean_fif.info['ch_names'] == raw_clean_fif.info['ch_names'] and filtered_clean_fif.info['bads'] == raw_clean_fif.info['bads']:
    
    # load filtered data
    filtered_clean_fif.load_data()
    
    # set filtered reference 
    bp_filt_fif = mne.set_bipolar_reference(filtered_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)
    
    # clear var
    bp_filt_fif = []
    filtered_clean_fif = []
    
    # load raw data
    raw_clean_fif.load_data()
    
    # set raw reference 
    bp_raw_fif = mne.set_bipolar_reference(raw_clean_fif, anode = anode, cathode = cathode)
    
    # save
    bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)

    
    # clear var
    bp_raw_fif = []
    raw_clean_fif = []    
    
    

Reading 0 ... 2998199  =      0.000 ...  1499.100 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=143, n_times=2998200
    Range : 0 ... 2998199 =      0.000 ...  1499.100 secs
Ready.
Added the following bipolar channels:
AL1 -AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL6, CL6-CL7, CL7-CL8, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, DL12-DL13, DL13-DL14, DL14-DL15, DL15-DL16, GL1-GL2, GL2-GL3, GL3-GL4, GL4-GL5, GL5-GL6, GL6-GL7, GL7-GL8, GL8-GL9, GL9-GL10, GL10-GL11, GL11-GL12, GL12-GL13, GL13-GL14, GL14-GL15, GL15-GL16, HL1-HL2, HL2-HL3, HL3-HL4, HL4-HL5, HL5-HL6, HL6-HL7, HL7-HL8, HL8-HL9, HL9-HL10, HL10-HL11, HL11-HL12, IL8-IL9, IL9-IL10

/tmp/ipykernel_9863/1336304392.py:12: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filt_fif.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif
[done]
Reading 0 ... 2998199  =      0.000 ...  1499.100 secs...
sEEG channel type selected for re-referencing
Creating RawArray with float64 data, n_channels=143, n_times=2998200
    Range : 0 ... 2998199 =      0.000 ...  1499.100 secs
Ready.
Added the following bipolar channels:
AL1 -AL2, AL2-AL3, AL3-AL4, AL4-AL5, AL5-AL6, AL6-AL7, AL7-AL8, AL8-AL9, AL9-AL10, AL10-AL11, AL11-AL12, AL12-AL13, AL13-AL14, BL1-BL2, BL2-BL3, BL3-BL4, BL4-BL5, BL5-BL6, BL6-BL7, BL7-BL8, BL8-BL9, BL9-BL10, BL10-BL11, BL11-BL12, BL12-BL13, BL13-BL14, CL1-CL2, CL2-CL3, CL3-CL4, CL4-CL6, CL6-CL7, CL7-CL8, DL1-DL2, DL2-DL3, DL3-DL4, DL4-DL5, DL5-DL6, DL6-DL7, DL7-DL8, DL8-DL9, DL9-DL10, DL10-DL11, DL11-DL12, DL12-DL13, 

/tmp/ipykernel_9863/1336304392.py:25: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_raw_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_raw_fif.save(f"{preproc_data_dir}/{subject}_bp_raw_clean_data.fif", overwrite = True)


Overwriting existing file.
Writing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_raw_clean_data-1.fif
Closing /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_raw_clean_data.fif
[done]


In [ ]:
# Visualize it #

# bp_raw_fif.plot(events=events, color='b', bad_color = 'cyan', n_channels = 1, clipping = None, event_color = 'r')

## Epoching the data 

### Onset

In [3]:
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')


Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.


/tmp/ipykernel_6267/644633114.py:1: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]


In [7]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 1, tmin = -4, tmax = 12, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.


/tmp/ipykernel_9863/1463149468.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
240 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 32001 original time points ...
Splitting into 3 parts
Loading data for 80 events and 32001 original time points ...


/tmp/ipykernel_9863/1463149468.py:18: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_clean_pres-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_pres-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 80 events and 32001 original time points ...
Loading data for 80 events and 32001 original time points ...


### Trial End

In [9]:
## Epoching the raw data ##

# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
events = mne.find_events(bp_filtered_fif, output='step', consecutive = False, stim_channel='STI')

# create events
events = events[0:-1]

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
epoched_data = mne.Epochs(bp_filtered_fif, events, 
                          event_id = 0, tmin = -5, tmax = 5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# save the data
epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')

Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.


/tmp/ipykernel_9863/875914644.py:4: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")


480 events found
Event IDs: [0 1]
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Overwriting existing file.
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts


/tmp/ipykernel_9863/875914644.py:19: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_clean_end-locked_ieeg.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epoched_data.save(f"{preproc_data_dir}/{subject}_bp_clean_end-locked_ieeg.fif", overwrite = True, split_size = '1.9GB')


Loading data for 120 events and 20001 original time points ...
Loading data for 119 events and 20001 original time points ...


### Last Away

In [10]:
## Load Behavioral Data ##
last_away_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_last_away_events.csv")
last_away_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,17200,0,1
1,1,3,26600,0,1
2,3,12,48800,0,1
3,4,6,61900,0,1
4,5,18,74000,0,1
...,...,...,...,...,...
192,233,7,2724700,0,1
193,234,12,2733800,0,1
194,235,20,2744800,0,1
195,237,8,2774200,0,1


In [11]:
# create events
last_away_events = last_away_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [12]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
    
# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]

# epoch the data
last_away_epochs = mne.Epochs(bp_filtered_fif, last_away_events, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.
Not setting metadata
197 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 99 events and 20001 original time points ...


/tmp/ipykernel_9863/678285560.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9863/678285560.py:13: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_last_away_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  last_away_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_last_away_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 98 events and 20001 original time points ...


### First Dot

In [13]:
## Load Behavioral Data ##
first_dot_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_dot_events.csv")
first_dot_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,12900,0,1
1,2,5,32900,0,1
2,3,12,46800,0,1
3,4,6,59900,0,1
4,5,18,70400,0,1
...,...,...,...,...,...
227,234,12,2733400,0,1
228,235,20,2742500,0,1
229,236,10,2754600,0,1
230,237,8,2772100,0,1


In [14]:
# create events
first_dot_data = first_dot_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [15]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_dot_epochs = mne.Epochs(bp_filtered_fif, first_dot_data, 
                          event_id = 1, tmin = -3, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.
Not setting metadata
232 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 18001 original time points ...
Splitting into 2 parts
Loading data for 116 events and 18001 original time points ...


/tmp/ipykernel_9863/4126832509.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9863/4126832509.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_first_dot_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_dot_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_dot_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 116 events and 18001 original time points ...


## First Move

In [16]:
## Load Behavioral Data ##
first_move_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_first_move_events.csv")
first_move_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,0,13,12700,0,1
1,1,3,26600,0,1
2,2,5,32700,0,1
3,3,12,44200,0,1
4,4,6,59100,0,1
...,...,...,...,...,...
234,234,12,2732800,0,1
235,235,20,2741700,0,1
236,236,10,2753800,0,1
237,237,8,2770300,0,1


In [17]:
# create events
first_move_data = first_move_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [18]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
first_move_epochs = mne.Epochs(bp_filtered_fif, first_move_data, 
                          event_id = 1, tmin = -5, tmax =5, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.
Not setting metadata
239 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Splitting into 2 parts
Loading data for 120 events and 20001 original time points ...


/tmp/ipykernel_9863/3347570552.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9863/3347570552.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_first_move_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  first_move_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_first_move_events.fif", overwrite = True, split_size = '1.9GB')


Loading data for 119 events and 20001 original time points ...


## Ghost Attack

In [19]:
## Load Behavioral Data ##
ghost_attack_data = pd.read_csv(f"{raw_data_dir}/behave/{subject}_attack_events.csv")
ghost_attack_data

,neural_trial_numeric,TrialType,sample,sample_before,event
0,2,5,35000,0,1
1,3,12,48500,0,1
2,8,7,110000,0,1
3,9,4,119700,0,1
4,14,11,170800,0,1
...,...,...,...,...,...
64,216,7,2531900,0,1
65,221,13,2590900,0,1
66,228,3,2667900,0,1
67,233,7,2724700,0,1


In [20]:
# create events
ghost_attack_data = ghost_attack_data[['sample', 'sample_before', 'event']].copy().to_numpy()


In [21]:
# load data 
bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")

# only epoch good channels
channels_of_interest =  [x for x in bp_filtered_fif.info['ch_names'] if x not in bp_filtered_fif.info['bads']]


# epoch the data
ghost_attack_epochs = mne.Epochs(bp_filtered_fif, ghost_attack_data, 
                          event_id = 1, tmin = -4, tmax =6, 
                          baseline = None, picks = channels_of_interest,
                             reject_by_annotation = False)
# # save the data
ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')


Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif...
    Range : 0 ... 2189999 =      0.000 ...  1094.999 secs
Ready.
Opening raw data file /home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data-1.fif...
    Range : 2190000 ... 2998199 =   1095.000 ...  1499.100 secs
Ready.
Not setting metadata
69 matching events found
No baseline correction applied
0 projection items activated
Loading data for 1 events and 20001 original time points ...
Loading data for 69 events and 20001 original time points ...


/tmp/ipykernel_9863/2681655913.py:2: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_data.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  bp_filtered_fif = mne.io.Raw(f"{preproc_data_dir}/{subject}_bp_filtered_clean_data.fif")
/tmp/ipykernel_9863/2681655913.py:14: RuntimeWarning: This filename (/home/brooke/pacman/preprocessing/BJH041/ieeg/BJH041_bp_filtered_clean_ghost_attack_events.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  ghost_attack_epochs.save(f"{preproc_data_dir}/{subject}_bp_filtered_clean_ghost_attack_events.fif", overwrite = True, split_size = '1.9GB')
